In [1]:
import os
os.listdir()

['uniprot-all.fasta', 'script.py', 'scripty.ipynb']

In [11]:
import sys
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import re

# check if command line arguments are provided
if len(sys.argv) < 2:
    print("USAGE: python3 related_IEP_MW.py <IEP lower limit (pH)> <IEP upper limit (pH)> <MW lower limit (Da)> <MW upper limit (Da)>")
    sys.exit(1)
else:
    # store command line arguments
    IEP_lower, IEP_upper = float(sys.argv[1]), float(sys.argv[2])
    MW_lower, MW_upper = float(sys.argv[3]), float(sys.argv[4])

# regerx to catch ambiguous protein sequences
pattern = re.compile(r"'[XBZJ]' is not a valid unambiguous letter for protein")

# parse protein sequences in FASTA file
for seq_record in SeqIO.parse("./data/uniprot-all.fasta", "fasta"):
    descr = seq_record.description
    try:
        # calculate molecular weight & isoelectric point
        protAnalysis = ProteinAnalysis(str(seq_record.seq))
        molWeight = protAnalysis.molecular_weight()
        isoPoint = protAnalysis.isoelectric_point()

    except ValueError as error:
        # skip unambiguous protein sequences
        if pattern.match(str(error)):
            continue
        else:
            raise
    
    # filter and return results
    if (MW_lower <= molWeight <= MW_upper) and (IEP_lower <= isoPoint <= IEP_upper):
        print(
            "Protein: {}\nMolecular Weight: {}\tIsoelectric Point: {}\n".format(
                descr, round(molWeight, 3), round(isoPoint, 3)))

In [4]:
# regex that matches: 'X' is not a valid unambiguous letter for protein but X can be (X,B,Z,J)

import re
pattern = re.compile(r"'[XBZJ]' is not a valid unambiguous letter for protein")

SeqRecord(seq=Seq('MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVV...GEN'), id='sp|P31946|1433B_HUMAN', name='sp|P31946|1433B_HUMAN', description='sp|P31946|1433B_HUMAN 14-3-3 protein beta/alpha OS=Homo sapiens GN=YWHAB PE=1 SV=3', dbxrefs=[])